In [ ]:
from cosapp.systems import System

from cpu.systems import CPU, HeatExchanger, Fan, FanController

            
class CpuSystem(System):

    def setup(self):

        exchanger = self.add_child(HeatExchanger("exchanger"))
        cpu = self.add_child(CPU("cpu"), pulling=["T"])
        controller = self.add_child(FanController("controler"))
        fan = self.add_child(Fan("fan"))
        
        self.connect(cpu.inwards, exchanger.inwards, ["T"])
        self.connect(fan.fl_out, exchanger.fl_in)
        self.connect(controller.outwards, fan.inwards, ["tension"])
        self.connect(cpu.inwards, controller.inwards, ["T"])
        self.connect(exchanger.outwards, cpu.inwards, ["heat_flow"])
        
        self.add_design_method("exchanger_surface").add_equation("exchanger.heat_flow == cpu.tdp").add_unknown("exchanger.surface")

In [ ]:
from cosapp.drivers import RungeKutta, NonLinearSolver
from cosapp.recorders import DataFrameRecorder

cpu = CpuSystem("cpu")

design = cpu.add_driver(NonLinearSolver('solver', max_iter=10, factor=1.0))
design.extend(cpu.design_methods["exchanger_surface"])
design.runner.set_values({"fan.T_air": 313.15, "T": 363.15, "cpu.usage": 100.})

time_driver = cpu.add_driver(RungeKutta(order=3, history = True))
solver = time_driver.add_child(NonLinearSolver('solver', max_iter=10, factor=1.0))

time_driver.time_interval = (0, 30)
time_driver.dt = 0.5
rec = time_driver.add_recorder(DataFrameRecorder(includes=['cpu.T', 'cpu.power', 'cpu.usage', 'cpu.heat_flow', 'exchanger.heat_flow', 'exchanger.h', 
                                                'exchanger.surface', 'fan.tension', 'fan.fl_out.mass_flow', 'controler.T', 'controler.tension'], hold=False), period=0.5)

# Define a simulation scenario
time_driver.set_scenario(
    init = {'T': cpu.fan.T_air},
)

cpu.run_drivers()

In [ ]:
data = rec.export_data()
data

In [ ]:
import plotly.graph_objs as go
import numpy as np


time = np.asarray(data['time'])
go.Figure(
    data = [
        go.Scatter(
            x = time,
            y = data['cpu.T'],
            mode = 'markers',
            marker_color = "blue",
            name = 'cpu.T',
            yaxis = "y1",
        ),
        go.Scatter(
            x = time,
            y = data['fan.tension'],
            mode = 'markers',
            marker_color = "red",
            name = 'fan.tension',
            yaxis = "y2",
        ),
    ],
    layout = go.Layout(
        title = "CPU temperature",
        width = 1500,
        height = 600,
        xaxis = {'title': 'time'},
        yaxis = {'title': 'temperature'},
        yaxis2 = {'title': 'tension', 'side': "right", "overlaying": "y"},
        hovermode = 'x',
    )
)

In [ ]:
from cosapp_lab.widgets import SysExplorer

# cpu.drivers.pop("solver")
a = SysExplorer(cpu, template = "cpu_lab.json", anchor = "tab-after")